In [3]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [4]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models/mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models/mod_Human1_Human1.xml")
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [5]:
mod_RPE_PR

Name,RPE_PR
Memory address,22f3fdeb1f0
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR13082_RPE - 1.0*MAR13082_RPE_reverse_4a4f4
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [5]:
mod_Human1_Human1

Name,HumanGEM_HumanGEM
Memory address,1b7ab1e3f70
Number of metabolites,16554
Number of reactions,29057
Number of genes,3067
Number of groups,142
Objective expression,1.0*MAR13082_RPE - 1.0*MAR13082_RPE_reverse_4a4f4
Compartments,"c_RPE, e_RPE, l_RPE, r_RPE, m_RPE, x_RPE, g_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, m_PR, x_PR, r_PR, l_PR, g_PR, i_PR"


In [3]:
# modify models
# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set objective function (PR ATP DM)
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod.objective = 'MAR03964'

FIRST SET OF ESTIMATES FOR SIMULATIONS						SECOND SET OF ESTIMATES FOR SIMULATIONS				
CC --> RPE			A			CC --> RPE			A	B
dark	oxygen (in)	pmol/s/mm2 	2.20			dark	oxygen (in)	pmol/s/mm2 	3.51	3.51
dark	glucose (in)	pmol/s/mm2 	2.62			dark	glucose (in)	pmol/s/mm2 	4.18	4.18
dark	lactate (out)	pmol/s/mm2 	4.54			dark	lactate (out)	pmol/s/mm2 	7.25	7.25
light	oxygen (in)	pmol/s/mm2 	1.84			light	oxygen (in)	pmol/s/mm2 	2.94	2.61
light	glucose (in)	pmol/s/mm2 	1.37			light	glucose (in)	pmol/s/mm2 	2.18	1.93
light	lactate (out)	pmol/s/mm2 	2.81			light	lactate (out)	pmol/s/mm2 	4.48	2.95
										
RPE --> PR						RPE --> PR				
dark	oxygen (in)	pmol/s/mm2 	1.90			dark	oxygen (in)	pmol/s/mm2 	3.21	3.21
dark	glucose (in)	pmol/s/mm2 				dark	glucose (in)	pmol/s/mm2 		
dark	lactate (out)	pmol/s/mm2 				dark	lactate (out)	pmol/s/mm2 		
light	oxygen (in)	pmol/s/mm2 	1.59			light	oxygen (in)	pmol/s/mm2 	2.31	2.31
light	glucose (in)	pmol/s/mm2 				light	glucose (in)	pmol/s/mm2 		
light	lactate (out)	pmol/s/mm2 				light	lactate (out)	pmol/s/mm2 		


In [4]:
from src.analysis import create_permutation_dicts
# MAR09048_RPE = oxygen exchange
# MAR09034_RPE = glucose exchange
# MAR09135_RPE = lactate exchange
# MAR04896_PR_RPE =  O2[c_PR] <=> O2[e_RPE_PR]

dark_old_labels = create_permutation_dicts({'oxygen':['limited oxygen (2.2)'],\
                      'glucose':['limited glucose (2.6)'],\
                      'lactate':['lactate efflux > 4.53'],\
                      'oxygen RPE -> PR':['fixed (1.9)','unlimited']})
dark_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(-2.2,1000)],\
                      'MAR09034_RPE':[(-2.62,1000)],\
                      'MAR09135_RPE':[(4.53,1000)],\
                      'MAR04896_PR_RPE':[(-1.9,-1.9),(-1000,1000)]})

dark_new_labels = create_permutation_dicts({'oxygen':['limited oxygen (3.51)'],\
                      'glucose':['limited glucose (4.18)'],\
                      'lactate':[ 'lactate efflux > 7.25'],\
                      'oxygen RPE -> PR':['fixed (3.21)','unlimited']})
dark_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(-3.51,1000)],\
                      'MAR09034_RPE':[(-4.18,1000)],\
                      'MAR09135_RPE':[(7.25,1000)],\
                      'MAR04896_PR_RPE':[(-3.21,-3.21),(-1000,1000)]})

light_old_labels = create_permutation_dicts({'oxygen':['limited oxygen (1.84)'],\
                      'glucose':['limited glucose (1.37)'],\
                      'lactate':['lactate efflux > 2.81'],\
                      'oxygen RPE -> PR':['fixed (1.59)','unlimited']})
light_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(-1.84,1000)],\
                      'MAR09034_RPE':[ (-1.37,1000)],\
                      'MAR09135_RPE':[ (2.81,1000)],\
                      'MAR04896_PR_RPE':[(-1.59,-1.59),(-1000,1000)]})

light_new_labels = create_permutation_dicts({'oxygen':['limited oxygen (2.61)'],\
                      'glucose':['limited glucose (1.93)'],\
                      'lactate':['lactate efflux > 2.95'],\
                      'oxygen RPE -> PR':['fixed (2.31)','unlimited']})
light_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(-2.61,1000)],\
                      'MAR09034_RPE':[ (-1.93,1000)],\
                      'MAR09135_RPE':[ (2.95,1000)],\
                      'MAR04896_PR_RPE':[(-2.31,-2.31),(-1000,1000)]})

In [5]:
def open_blood_exchange(mod,br, glucose_level):  
    br_updated = br.copy()
    br_updated.update((x, y * glucose_level) for x, y in br_updated.items())
    for key in br:
        mod.reactions.get_by_id(key).bounds = (br_updated[key],1000)# allow influx for RPE exchange reactions (blood exchange)
        
def close_blood_exchange(mod,br):  
    for key in br:
        mod.reactions.get_by_id(key).bounds = (0,1000)# allow influx for RPE exchange reactions (blood exchange)

In [6]:
blood_ratio = {df['id'][i]:df['ratio [glucose]/[x]'] [i] for i in range(len(df['id']))}
glucose_level = -2.622342019
# blood_ratio.update((x, y*glucose_level) for x, y in blood_ratio.items())
blood_ratio

{'MAR09034_RPE': 1.0,
 'MAR09063_RPE': 0.12618432385874245,
 'MAR09061_RPE': 0.090869939707149,
 'MAR09067_RPE': 0.05555555555555555,
 'MAR09041_RPE': 0.050818260120585705,
 'MAR09046_RPE': 0.05017226528854436,
 'MAR09068_RPE': 0.0409130060292851,
 'MAR09044_RPE': 0.03574504737295435,
 'MAR09069_RPE': 0.03229974160206718,
 'MAR09040_RPE': 0.026485788113695088,
 'MAR09038_RPE': 0.023040482342807925,
 'MAR09066_RPE': 0.017226528854435832,
 'MAR09064_RPE': 0.016731266149870805,
 'MAR09043_RPE': 0.01309216192937123,
 'MAR09065_RPE': 0.01119724375538329,
 'MAR09045_RPE': 0.009474590869939707,
 'MAR09071_RPE': 0.009323858742463394,
 'MAR09062_RPE': 0.008828596037898364,
 'MAR09042_RPE': 0.005383290267011197,
 'MAR09070_RPE': 0.0032730404823428077,
 'MAR11961_RPE': 0.00014857881136950905,
 'MAR09039_RPE': 0.014233419465977604}

In [7]:
# modify models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964_PR' # ATP demand in PR
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) # close all exchange reactions
    open_blood_exchange(m,blood_ratio,glucose_level)# allow influx for RPE exchange reactions (blood exchange)

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [8]:
ATP_mets = [m.id for m in mod_RPE_PR.metabolites if m.name=='ATP']

In [9]:
ATP_mets

['MAM01371c_RPE',
 'MAM01371g_RPE',
 'MAM01371l_RPE',
 'MAM01371m_RPE',
 'MAM01371n_RPE',
 'MAM01371x_RPE',
 'MAM01371r_RPE',
 'MAM01371e_RPE',
 'MAM01371e_RPE_PR',
 'MAM01371c_PR',
 'MAM01371m_PR',
 'MAM01371l_PR',
 'MAM01371r_PR',
 'MAM01371x_PR',
 'MAM01371g_PR']

In [10]:
l_ATP_all=list()
for r in mod_RPE_PR.reactions:
    for m in ATP_mets:
        if m in r.reaction:
            #print(r.id)
            l_ATP_all.append(r)
len(l_ATP_all)

1045

In [11]:
l_ATP_generate = list()
for r in mod_RPE_PR.reactions: 
    for p in r.products:
        for m in ATP_mets:
            if m == p.id:
                l_ATP_generate.append(r)
len(l_ATP_generate)

329

In [12]:
l_ATP_use = list()
for r in mod_RPE_PR.reactions: 
    for r2 in r.reactants:
        for m in ATP_mets:
            if m == r2.id:
                l_ATP_use.append(r)
len(l_ATP_use)

714

In [13]:
714+329

1043

In [79]:
def fba_ATP_rxns(mod, boundary_dicts, rxns, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df, add_compartment2rxn
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    conditions = dict()
    ovs = dict() # objective values
    f = dict() # fba fluxes
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
    rxn_info = dict()
    boundary_dicts_df= pd.DataFrame()
    boundary_labels_df= pd.DataFrame()
    
    i = 0
    ii = 0 
    
    with mod as m:
            
        # run analysis for every dict in list
        for d in boundary_dicts:

            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
            
            for r in rxns:
                m.objective = r.id
                # run analysis for all ATP consumption reactions
                fba = m.optimize() # fba
                label = r.id
                
                rxn_info[i] =  [r.id,r.name,add_compartment2rxn(r)]

                # objective values
                ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
                print(add_compartment2rxn(r))

                if fba.status == 'optimal':
                    try:
                        # write analysis info / results into dicts

                        # uptake / secretion
                        uptake_summary = m.summary().uptake_flux
                        secretion_summary = m.summary().secretion_flux
                        uptake[i] = pd.DataFrame(uptake_summary['flux'])
                        secretion[i] = pd.DataFrame(secretion_summary['flux'])

                        # all fluxes
                        f[label] = fba.to_frame()['fluxes']

                        pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T
                        # info
                        boundary_dicts_df = pd.concat([boundary_dicts_df,pd.DataFrame(pd.DataFrame(boundary_dicts).iloc[ii]).T])
                        boundary_labels_df = pd.concat([boundary_labels_df,pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T])
                    except UserWarning(...):
                        continue
                i = i + 1
            ii= ii + 1

    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)  
    
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    
    rxn_info_df = pd.DataFrame(rxn_info).T.rename(\
                  columns={0:'objective rxn id',\
                           1:'objective rxn name',\
                           2:'objective reaction'})
    
    ovs_df = pd.concat([rxn_info_df,\
                        boundary_dicts_df.reset_index(),\
                        boundary_labels_df.reset_index(),\
                        pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T]\
                       ,axis=1).drop('index', axis=1)

    
    # uptake / secretion dfs

    uptake_df = pd.concat(uptake,axis=1)
    secretion_df = pd.concat(secretion,axis=1)
    
    secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in secretion_df.index],\
                               columns=['metabolite'],index= secretion_df.index)
    uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in uptake_df.index],\
                               columns=['metabolite'],index= uptake_df.index)
    
    uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
    secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                  
    # sort uptake / secretion dfs
    uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
    secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)

    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('FBA_' + results_name + '_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
        
    return [ovs_df, uptake_df, secretion_df, fluxes_df]

In [15]:
a = fba_ATP_rxns(mod_RPE_PR, dark_new_estimates, l_ATP_generate, dark_new_labels,results_name='ATP_generating_reactions_dark_new_CS')

ADP[c_RPE] + H+[c_RPE] + PEP[c_RPE] --> ATP[c_RPE] + pyruvate[c_RPE]
1,3-bisphospho-D-glycerate[c_RPE] + ADP[c_RPE] <=> 3-phospho-D-glycerate[c_RPE] + ATP[c_RPE]
ADP[c_RPE] + H+[c_RPE] + ribose-5-phosphate[c_RPE] <=> ATP[c_RPE] + ribose[c_RPE]
6-phospho-D-gluconate[c_RPE] + ADP[c_RPE] + H+[c_RPE] <=> ATP[c_RPE] + D-gluconic acid[c_RPE]
ADP[c_RPE] + H+[c_RPE] + sedoheptulose-1,7-bisphosphate[c_RPE] <=> ATP[c_RPE] + sedoheptulose-7-phosphate[c_RPE]
2.0 ADP[c_RPE] <=> AMP[c_RPE] + ATP[c_RPE]
2.0 ADP[m_RPE] <=> AMP[m_RPE] + ATP[m_RPE]
ADP[c_RPE] + GDP[c_RPE] <=> ATP[c_RPE] + GMP[c_RPE]
ADP[m_RPE] + GDP[m_RPE] <=> ATP[m_RPE] + GMP[m_RPE]
ADP[c_RPE] + dADP[c_RPE] <=> ATP[c_RPE] + dAMP[c_RPE]
ADP[c_RPE] + dGDP[c_RPE] <=> ATP[c_RPE] + dGMP[c_RPE]
ADP[c_RPE] + GTP[c_RPE] <=> ATP[c_RPE] + GDP[c_RPE]
ADP[m_RPE] + GTP[m_RPE] <=> ATP[m_RPE] + GDP[m_RPE]
ADP[c_RPE] + dATP[c_RPE] <=> ATP[c_RPE] + dADP[c_RPE]
ADP[c_RPE] + ITP[c_RPE] <=> ATP[c_RPE] + IDP[c_RPE]
ADP[m_RPE] + dATP[m_RPE] --> ATP[m_RPE] +

C:\Users\prins\AppData\Local\Temp\ipykernel_6084\2757991994.py:97: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\2757991994.py:98: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\2757991994.py:107: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [36]:
b = fba_ATP_rxns(mod_RPE_PR, dark_new_estimates, l_ATP_use, dark_new_labels,results_name='ATP_using_reactions_dark_new_CS')

ATP[c_RPE] + fructose-6-phosphate[c_RPE] --> ADP[c_RPE] + fructose-1,6-bisphosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + glucose[c_RPE] --> ADP[c_RPE] + glucose-6-phosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + glyceraldehyde[c_RPE] --> ADP[c_RPE] + GAP[c_RPE] + H+[c_RPE]
ATP[c_RPE] + fructose-6-phosphate[c_RPE] --> ADP[c_RPE] + fructose-2,6-bisphosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + fructose[c_RPE] --> ADP[c_RPE] + fructose-6-phosphate[c_RPE] + H+[c_RPE]
6-deoxy-L-galactose[c_RPE] + ATP[c_RPE] <=> ADP[c_RPE] + H+[c_RPE] + L-fucose-1-phosphate[c_RPE]
ATP[c_RPE] + mannose[c_RPE] --> ADP[c_RPE] + H+[c_RPE] + mannose-6-phosphate[c_RPE]
ATP[c_RPE] + ribose-5-phosphate[c_RPE] --> AMP[c_RPE] + H+[c_RPE] + PRPP[c_RPE]
ATP[c_RPE] --> cAMP[c_RPE] + PPi[c_RPE]
ATP[c_RPE] + glutamine[c_RPE] + H2O[c_RPE] + xanthosine-5-phosphate[c_RPE] --> AMP[c_RPE] + glutamate[c_RPE] + GMP[c_RPE] + 2.0 H+[c_RPE] + PPi[c_RPE]
ATP[c_RPE] + NH3[c_RPE] + xanthosine-5-phosphate[c_RPE] --> AMP[c_RPE] + GMP[c_RPE] + H+[c_RPE] +

Infeasible: None (infeasible).

In [78]:
        mod=mod_RPE_PR
        boundary_dicts=dark_new_estimates
        rxns= l_ATP_use
        boundary_labels= dark_new_labels
        results_name='TEST_ATP_using_reactions_dark_new_CS'

        # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
        import pandas as pd
        from datetime import datetime
        from src.get_info import make_rxn_df, make_compact_rxn_df, add_compartment2rxn
        from datetime import datetime

        #  create empty dicts
        bounds = dict() # changed model bounds
        conditions = dict()
        ovs = dict() # objective values
        f = dict() # fba fluxes
        uptake = dict() # uptake fluxes
        secretion = dict() # secretion fluxes
        rxn_info = dict()
        boundary_dicts_df= pd.DataFrame()
        boundary_labels_df= pd.DataFrame()

        i = 0
        ii = 0 
    
        mod = m
        
        # run analysis for every dict in list
        for d in boundary_dicts:

            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
            
            for r in rxns:
                m.objective = r.id
                # run analysis for all ATP consumption reactions
                fba = m.optimize() # fba
                label = r.id
                
                rxn_info[i] =  [r.id,r.name,add_compartment2rxn(r)]

                # objective values
                ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
                print(add_compartment2rxn(r))

                if fba.status == 'optimal':
                    try:
                        # write analysis info / results into dicts

                        # uptake / secretion
                        uptake_summary = m.summary().uptake_flux
                        secretion_summary = m.summary().secretion_flux
                        uptake[i] = pd.DataFrame(uptake_summary['flux'])
                        secretion[i] = pd.DataFrame(secretion_summary['flux'])

                        # all fluxes
                        f[label] = fba.to_frame()['fluxes']

                        pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T
                        # info
                        boundary_dicts_df = pd.concat([boundary_dicts_df,pd.DataFrame(pd.DataFrame(boundary_dicts).iloc[ii]).T])
                        boundary_labels_df = pd.concat([boundary_labels_df,pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T])
                    except:
                        continue
                i = i + 1
            ii= ii + 1

    
        # prepare dfs for excel sheet
        model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
                 index=['id','name','compartments','annotation'],columns = ['model'])

        rxn_df = make_rxn_df(m)   
        compact_rxn_df = make_compact_rxn_df(m)  

        met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
                 index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])

        rxn_info_df = pd.DataFrame(rxn_info).T.rename(\
                      columns={0:'objective rxn id',\
                               1:'objective rxn name',\
                               2:'objective reaction'})

        ovs_df = pd.concat([rxn_info_df,\
                            boundary_dicts_df.reset_index(),\
                            boundary_labels_df.reset_index(),\
                            pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T]\
                           ,axis=1).drop('index', axis=1)


        # uptake / secretion dfs

        uptake_df = pd.concat(uptake,axis=1)
        secretion_df = pd.concat(secretion,axis=1)

        secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in secretion_df.index],\
                                   columns=['metabolite'],index= secretion_df.index)
        uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in uptake_df.index],\
                                   columns=['metabolite'],index= uptake_df.index)

        uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
        secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  

        # sort uptake / secretion dfs
        uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
        secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)


        # fluxes df
        fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
        fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
        # sort table on absolute flux size 
        fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        # select internal fluces only
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]

        # date stamp
        datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   

        # write excel file
        with pd.ExcelWriter('FBA_' + results_name + '_' + datestr + '.xlsx') as writer:  
            model_info.to_excel(writer, sheet_name = 'model')
            rxn_df.to_excel(writer, sheet_name = 'reactions')
            met_df.to_excel(writer, sheet_name = 'metabolites')
            ovs_df.to_excel(writer, sheet_name = 'objective_values')  
            uptake_df.to_excel(writer, sheet_name = 'uptake')    
            secretion_df.to_excel(writer, sheet_name = 'secretion')    
            fluxes_df.to_excel(writer, sheet_name = 'fluxes')     


ATP[c_RPE] + fructose-6-phosphate[c_RPE] --> ADP[c_RPE] + fructose-1,6-bisphosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + glucose[c_RPE] --> ADP[c_RPE] + glucose-6-phosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + glyceraldehyde[c_RPE] --> ADP[c_RPE] + GAP[c_RPE] + H+[c_RPE]
ATP[c_RPE] + fructose-6-phosphate[c_RPE] --> ADP[c_RPE] + fructose-2,6-bisphosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + fructose[c_RPE] --> ADP[c_RPE] + fructose-6-phosphate[c_RPE] + H+[c_RPE]
6-deoxy-L-galactose[c_RPE] + ATP[c_RPE] <=> ADP[c_RPE] + H+[c_RPE] + L-fucose-1-phosphate[c_RPE]
ATP[c_RPE] + mannose[c_RPE] --> ADP[c_RPE] + H+[c_RPE] + mannose-6-phosphate[c_RPE]
ATP[c_RPE] + ribose-5-phosphate[c_RPE] --> AMP[c_RPE] + H+[c_RPE] + PRPP[c_RPE]
ATP[c_RPE] --> cAMP[c_RPE] + PPi[c_RPE]
ATP[c_RPE] + glutamine[c_RPE] + H2O[c_RPE] + xanthosine-5-phosphate[c_RPE] --> AMP[c_RPE] + glutamate[c_RPE] + GMP[c_RPE] + 2.0 H+[c_RPE] + PPi[c_RPE]
ATP[c_RPE] + NH3[c_RPE] + xanthosine-5-phosphate[c_RPE] --> AMP[c_RPE] + GMP[c_RPE] + H+[c_RPE] +

C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3853372845.py:104: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3853372845.py:105: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3853372845.py:114: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [80]:
d = fba_ATP_rxns(mod_Human1_Human1, dark_new_estimates, l_ATP_generate, dark_new_labels,results_name='ATP_generating_reactions_dark_new_H1')

ADP[c_RPE] + H+[c_RPE] + PEP[c_RPE] --> ATP[c_RPE] + pyruvate[c_RPE]
1,3-bisphospho-D-glycerate[c_RPE] + ADP[c_RPE] <=> 3-phospho-D-glycerate[c_RPE] + ATP[c_RPE]
ADP[c_RPE] + H+[c_RPE] + ribose-5-phosphate[c_RPE] <=> ATP[c_RPE] + ribose[c_RPE]
6-phospho-D-gluconate[c_RPE] + ADP[c_RPE] + H+[c_RPE] <=> ATP[c_RPE] + D-gluconic acid[c_RPE]
ADP[c_RPE] + H+[c_RPE] + sedoheptulose-1,7-bisphosphate[c_RPE] <=> ATP[c_RPE] + sedoheptulose-7-phosphate[c_RPE]
2.0 ADP[c_RPE] <=> AMP[c_RPE] + ATP[c_RPE]
2.0 ADP[m_RPE] <=> AMP[m_RPE] + ATP[m_RPE]
ADP[c_RPE] + GDP[c_RPE] <=> ATP[c_RPE] + GMP[c_RPE]
ADP[m_RPE] + GDP[m_RPE] <=> ATP[m_RPE] + GMP[m_RPE]
ADP[c_RPE] + dADP[c_RPE] <=> ATP[c_RPE] + dAMP[c_RPE]
ADP[c_RPE] + dGDP[c_RPE] <=> ATP[c_RPE] + dGMP[c_RPE]
ADP[c_RPE] + GTP[c_RPE] <=> ATP[c_RPE] + GDP[c_RPE]
ADP[m_RPE] + GTP[m_RPE] <=> ATP[m_RPE] + GDP[m_RPE]
ADP[c_RPE] + dATP[c_RPE] <=> ATP[c_RPE] + dADP[c_RPE]
ADP[c_RPE] + ITP[c_RPE] <=> ATP[c_RPE] + IDP[c_RPE]
ADP[m_RPE] + dATP[m_RPE] --> ATP[m_RPE] +

C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3601613998.py:99: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3601613998.py:100: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3601613998.py:109: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [81]:
e = fba_ATP_rxns(mod_Human1_Human1, dark_new_estimates, l_ATP_use, dark_new_labels,results_name='ATP_using_reactions_dark_new_H1')

ATP[c_RPE] + fructose-6-phosphate[c_RPE] --> ADP[c_RPE] + fructose-1,6-bisphosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + glucose[c_RPE] --> ADP[c_RPE] + glucose-6-phosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + glyceraldehyde[c_RPE] --> ADP[c_RPE] + GAP[c_RPE] + H+[c_RPE]
ATP[c_RPE] + fructose-6-phosphate[c_RPE] --> ADP[c_RPE] + fructose-2,6-bisphosphate[c_RPE] + H+[c_RPE]
ATP[c_RPE] + fructose[c_RPE] --> ADP[c_RPE] + fructose-6-phosphate[c_RPE] + H+[c_RPE]
6-deoxy-L-galactose[c_RPE] + ATP[c_RPE] <=> ADP[c_RPE] + H+[c_RPE] + L-fucose-1-phosphate[c_RPE]
ATP[c_RPE] + mannose[c_RPE] --> ADP[c_RPE] + H+[c_RPE] + mannose-6-phosphate[c_RPE]
ATP[c_RPE] + ribose-5-phosphate[c_RPE] --> AMP[c_RPE] + H+[c_RPE] + PRPP[c_RPE]
ATP[c_RPE] --> cAMP[c_RPE] + PPi[c_RPE]
ATP[c_RPE] + glutamine[c_RPE] + H2O[c_RPE] + xanthosine-5-phosphate[c_RPE] --> AMP[c_RPE] + glutamate[c_RPE] + GMP[c_RPE] + 2.0 H+[c_RPE] + PPi[c_RPE]
ATP[c_RPE] + NH3[c_RPE] + xanthosine-5-phosphate[c_RPE] --> AMP[c_RPE] + GMP[c_RPE] + H+[c_RPE] +

C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3601613998.py:99: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3601613998.py:100: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_6084\3601613998.py:109: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [31]:
m.reactions.get_by_id('MAR04156_RPE')

Reaction identifier,MAR04156_RPE
Name,acetyl-CoA:carbon-dioxide ligase (ADP-forming)
Memory address,0x1d282850ee0
Stoichiometry,MAM01261c_RPE + MAM01371c_RPE + MAM02046c_RPE --> MAM01285c_RPE + MAM02039c_RPE + MAM02444c_RPE + MAM02751c_RPE acetyl-CoA + ATP + HCO3- --> ADP + H+ + malonyl-CoA + Pi
GPR,ENSG00000076555 or ENSG00000278540
Lower bound,0.0
Upper bound,1000.0
